In [1]:
version = 'v20250521'
import os
os.environ['USE_PYGEOS'] = '0'
import geopandas as gpd
import pandas as pd
from shapely.geometry import Point
import copy
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import GroupKFold,cross_val_predict
from skmap.misc import find_files, GoogleSheet, ttprint
import joblib
import numpy as np

## creat pixels

In [2]:
# import rasterio
# from rasterio.mask import mask
# import os
# os.environ['USE_PYGEOS'] = '0'
# import geopandas as gpd
# from shapely.geometry import Point
# import numpy as np
# import pandas as pd
# from skmap.misc import find_files, GoogleSheet, ttprint

# rtif = 'http://192.168.1.30:8333/ai4sh-landmasked/air.temp/clm_air.temp_era5.copernicus_mx_1km_200..200cm_2019.04.01..2019.04.30_eumap_epsg3035_v0.1.tif'

# ttprint('de')
# polys = gpd.read_file("./material/nuts_de_2021.gpkg")         
# with rasterio.open(rtif) as src:
#     # out_image → ndarray (bands, rows, cols)
#     # out_transform → affine transform for the masked array
#     out_image, out_transform = mask(src, polys.geometry, crop=True)
#     out_image = out_image[0]                    # assume one-band raster
#     crs = src.crs                               # keep raster CRS

# height, width = out_image.shape
# rows, cols = np.meshgrid(np.arange(height), np.arange(width), indexing='ij')
# xs, ys = rasterio.transform.xy(out_transform, rows, cols, offset='center')
# df = pd.DataFrame({
#     "x": np.array(xs).flatten(),
#     "y": np.array(ys).flatten(),
#     "value": out_image.flatten()
# })

# de = gpd.GeoDataFrame(
#         df,
#         geometry=[Point(x, y) for x, y in zip(df.x, df.y)],
#         crs=crs
# )

# ttprint('es')
# polys = gpd.read_file("./material/nuts_es_2021.gpkg")         
# with rasterio.open(rtif) as src:
#     out_image, out_transform = mask(src, polys.geometry, crop=True)
#     out_image = out_image[0]                    # assume one-band raster
#     crs = src.crs                               # keep raster CRS

# height, width = out_image.shape
# rows, cols = np.meshgrid(np.arange(height), np.arange(width), indexing='ij')
# xs, ys = rasterio.transform.xy(out_transform, rows, cols, offset='center')
# df = pd.DataFrame({
#     "x": np.array(xs).flatten(),
#     "y": np.array(ys).flatten(),
#     "value": out_image.flatten()
# })
# es = gpd.GeoDataFrame(
#         df,
#         geometry=[Point(x, y) for x, y in zip(df.x, df.y)],
#         crs=crs
# )

# ttprint('finish')

# de['nuts0'] = 'DE'
# es['nuts0'] = 'ES'

# agg_pixel = pd.concat([de,es])
# agg_pixel = agg_pixel.drop(columns=['geometry','value'])
# agg_pixel.to_parquet('./material/pixel_agg.pq')

## read back in the overlaid version

In [2]:
tgt = 'soc_log1p'
prop = 'soc'
# df = pd.read_parquet('./pixel_agg.pq')

covs = pd.read_csv(f'./metric/feature_selected_soc_{version}.txt', header=None)[0].tolist() 

df = pd.read_parquet(f'./material/agg.pixel.tmp_overlaid.2015_{version}.pq')
print(df.shape)
dff = df.dropna(subset=covs,how='any')
print(dff.shape)

(2638770, 393)
(1167802, 393)


## cast the models

In [3]:
# read in model
model = joblib.load(f'./model/model_rf.soc_ccc_{version}.joblib')
model.n_jobs = 90

# use all the points to train model
train = pd.read_parquet(f'./material/soc.topsoil_organized_{version}.pq')
train[tgt] = np.log1p(train[prop])
train = train.dropna(subset=covs+[tgt],how='any').reset_index(drop=True)

# fit
model.fit(train[covs], train[tgt])

# cast the model
import copy
from trees_rf import cast_tree_rf, cast_node_rf, pad_leaf_outputs_to_array
import warnings
warnings.filterwarnings("ignore", message="X has feature names, but")
ttprint('casting the model')
model_copy = copy.deepcopy(model)
modeln = cast_node_rf(model_copy, train[covs], train[tgt])
ttprint('finish casting')

[12:16:48] casting the model
[12:17:03] finish casting


## start prediction

In [4]:
# prediction
ttprint('start prediction')
y_pred = model.predict(dff[covs])
y_pred = np.expm1(y_pred)
dff[f'pred'] = y_pred
ttprint('finish prediction')

[12:17:03] start prediction
[12:17:04] finish prediction


/tmp/ipykernel_692879/1822879946.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dff[f'pred'] = y_pred


In [5]:
# do the predictions in chunck
chunk_size = 10000
num_chunks = int(np.ceil(len(dff) / chunk_size))

pred_std_chunks = []

ttprint(f'In total {num_chunks} chunks')

for i in range(num_chunks):
    start = i * chunk_size
    end = min((i + 1) * chunk_size, len(dff))
    ttprint(f'chunk {i + 1}/{num_chunks} ({start}:{end})--------')

    chunk = dff.iloc[start:end]

    ttprint('start predicting and padding')
    node_preds_chunk = modeln.predict(chunk[covs])
    nodes_chunk = pad_leaf_outputs_to_array(node_preds_chunk, pad_value=np.nan)
    nodes_chunk = np.expm1(nodes_chunk)
    
    ttprint('start calculating std')
    std_chunk = np.nanstd(nodes_chunk.T, axis=0)
    pred_std_chunks.append(std_chunk)

ttprint(f'finish all-------------------------------')

dff['pred_std'] = np.concatenate(pred_std_chunks)

dff.to_parquet(f'./material/pixel_agg.predicted_2015.{version}.pq')

[12:17:05] In total 117 chunks
[12:17:05] chunk 1/117 (0:10000)--------
[12:17:05] start predicting and padding
[12:17:53] start calculating std
[12:17:54] chunk 2/117 (10000:20000)--------
[12:17:54] start predicting and padding
[12:18:42] start calculating std
[12:18:42] chunk 3/117 (20000:30000)--------
[12:18:42] start predicting and padding
[12:19:31] start calculating std
[12:19:31] chunk 4/117 (30000:40000)--------
[12:19:31] start predicting and padding
[12:20:19] start calculating std
[12:20:19] chunk 5/117 (40000:50000)--------
[12:20:19] start predicting and padding
[12:21:08] start calculating std
[12:21:08] chunk 6/117 (50000:60000)--------
[12:21:08] start predicting and padding
[12:21:57] start calculating std
[12:21:57] chunk 7/117 (60000:70000)--------
[12:21:57] start predicting and padding
[12:22:46] start calculating std
[12:22:46] chunk 8/117 (70000:80000)--------
[12:22:46] start predicting and padding
[12:23:35] start calculating std
[12:23:35] chunk 9/117 (80000

/tmp/ipykernel_692879/2775512733.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dff['pred_std'] = np.concatenate(pred_std_chunks)


## join with the other years

In [2]:

dfo = pd.read_parquet(f'./material/pixel_agg.predicted_2015.{version}.pq')
dfo = dfo[['lon','lat','pred','pred_std']]
dfo =dfo.rename(columns={"pred":"pred_2015","pred_std":"pred_std_2015"})
geometry = [Point(xy) for xy in zip(dfo['lon'], dfo['lat'])]
dfo = gpd.GeoDataFrame(dfo, geometry=geometry, crs="EPSG:3035")


## aggregation

In [3]:
# point
from scipy.spatial import distance_matrix

# correlation function
def exponential_model(h, nugget, sill, range_):
    return nugget + (sill - nugget) * (1 - np.exp(-h / range_))

def correlation_function(h, nugget, sill, range_):
    # Calculate the variogram value at distance h
    gamma_h = exponential_model(h, nugget, sill, range_)
    # Calculate and return the correlation function value
    return (sill - gamma_h) / sill

params_es = np.array([0.74285565, 1.00000001, 5.99345285])
params_de = np.array([0.72415892,  1.00000001, 10.60109839])

In [12]:
from sklearn.linear_model import HuberRegressor, TheilSenRegressor

tsr = TheilSenRegressor(random_state=42)
df_base = dfo.copy()

sizes = [2.5, 10, 40, 200]

for isize in sizes:
    ttprint(isize, '-------------------------------')
    plg_id_col = f'id_{isize}'
    
    plg = gpd.read_file(f'./material/grid_all.{isize}km_agg.vlz.{version}.gpkg')
    plg = plg[['CNTR_CODE', 'var_2009', 'var_2018', 'mean_2009', 'mean_2018', 
               'change', 'signal', 'noise','snr', 'geometry']]
    df = gpd.sjoin(df_base, plg, how="inner", predicate="intersects")
    df = df.rename(columns={'index_right':f'id_{isize}','CNTR_CODE':'nuts0'})
    plg = plg.rename(columns={'signal':'signal_change','noise':'noise_change','snr':'snr_change'})
    
    group_ids = df[plg_id_col].dropna().unique().tolist()

    for id_val in group_ids:
        ipnt = df.loc[df[plg_id_col] == id_val]
        if ipnt.empty:
            continue
        
        params_vg = params_es if ipnt.iloc[0]['nuts0'] == 'ES' else params_de
 
        # distance
        coords = np.array([(geom.x, geom.y) for geom in ipnt.geometry])
        dist_matrix = distance_matrix(coords, coords) / 1000
        corr_matrix = correlation_function(dist_matrix, *params_vg)
        np.fill_diagonal(corr_matrix, 1.0)
        # variances
        std15 = ipnt['pred_std_2015'].to_numpy()
        var_matrix_2015 = np.outer(std15, std15) * corr_matrix
        n = len(ipnt)
        # aggregation
        agg_var_2015 = np.nansum(var_matrix_2015) / (n * n)
        m15 = ipnt['pred_2015'].mean()
        plg.loc[plg.index == id_val, 'var_2015'] = agg_var_2015
        plg.loc[plg.index == id_val, 'mean_2015'] = m15
        
        # linear trend fitting
        soc_p = plg.loc[plg.index == id_val, ['mean_2009','mean_2015','mean_2018']].values.squeeze()
        soc_std = np.sqrt(plg.loc[plg.index == id_val, ['var_2009','var_2015','var_2018']].values.squeeze())

        # fit linear slope
        X = np.array([0,6,9]) # 2009, 2015, 2018
        # # for observations
        # tsr.fit(X.reshape(-1, 1), np.array(soc_s))
        # slope_sample.append(tsr.coef_[0])
        # intercept_sample.append(tsr.intercept_)
        # score_sample.append(tsr.score(X.reshape(-1, 1), np.array(soc_s)))
        # # for prediction
        y = np.array(soc_p, dtype=float)
        mask = np.isfinite(y)  # keep only finite (non-NaN, non-inf) values
        if mask.sum() > 2:    # need at least 2 points to fit a line
            tsr.fit(X[mask].reshape(-1, 1), y[mask])
            tsr.fit(X.reshape(-1, 1), np.array(soc_p))
            # intercept_pred.append(tsr.intercept_)
            # score_pred.append(tsr.score(X.reshape(-1, 1), np.array(soc_p)))

            # shape becomes (n_realizations, n_time_steps), e.g., (100, 3)
            n_realizations = 100
            rt = np.random.normal(loc=soc_p, scale=soc_std, size=(n_realizations, 3))
            slope_r = []

            for i in range(100):  # number of Monte Carlo samples
                # Sample one realization (index) per time step
                # Choose a random index from 0 to 119 (assuming 120 realizations)
                sample_indices = np.random.randint(0, rt.shape[0], size=rt.shape[1])
                ss = np.array([rt[sample_indices[j], j] for j in range(rt.shape[1])])  # shape (n_time_steps,)

                tsr.fit(X.reshape(-1, 1), ss)
                slope_r.append(tsr.coef_[0])
                # score_r.append(tsr.score(X.reshape(-1, 1), ss))

            plg.loc[plg.index == id_val, 'noise_series'] = np.std(slope_r)
            plg.loc[plg.index == id_val, 'trend'] = tsr.coef_[0]

        else:
            plg.loc[plg.index == id_val, 'noise_series'] = np.nan
            plg.loc[plg.index == id_val, 'trend'] = np.nan
            
    # Merge back into plg
    plg['signal_series'] = plg['trend'].abs()
    plg['snr_series']  = plg['signal_series']/plg['noise_series']

    plg.to_file(f'./material/grid_all.{isize}km_agg.srs.{version}.gpkg')


[14:05:20] 2.5 -------------------------------
[20:36:12] 10 -------------------------------
[21:03:13] 40 -------------------------------
[21:06:20] 200 -------------------------------


In [14]:
plg.columns

Index(['CNTR_CODE', 'var_2009', 'var_2018', 'mean_2009', 'mean_2018', 'change',
       'signal_change', 'noise_change', 'snr_change', 'geometry', 'var_2015',
       'mean_2015', 'noise_series', 'trend', 'signal_series', 'snr_series'],
      dtype='object')